In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorboard.plugins.hparams import api as hp_api
import kerastuner
import numpy as np
import pandas as pd
import os
import json
import datetime
import dill

In [2]:
from tensorflow.keras.layers import (
    TimeDistributed, 
    Dense, 
    Conv1D, 
    MaxPooling1D, 
    Bidirectional, 
    LSTM, 
    Dropout
)

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [26]:
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "logs/" + timestamp
version_dir = "version/" + timestamp 

os.makedirs(log_dir)
os.makedirs(version_dir)
timestamp

'20200917-120410'

In [5]:
dataset_name = "SEG_AR"

In [6]:
static_params = {
    'PAST_HISTORY': 16,
    'FUTURE_TARGET': 8,
    'BATCH_SIZE': 1024,
    'BUFFER_SIZE': 200000,
    'EPOCHS': 500,
    'VOCAB_SIZE': 16293
 }

In [17]:
hparams = {
    "HP_LSTM_1_UNITS" : 128,
    "HP_LSTM_1_DROPOUT" : 0.0,
    "HP_LEARNING_RATE" : 1e-3,
}

In [8]:
def generate_timeseries(dataset, start_index, end_index, history_size, target_size):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        # Reshape data from (history_size,) to (history_size, n_feature)
        data.append(np.reshape(dataset[indices], (history_size, 1)))
        #data.append(dataset[indices])
        labels.append(np.reshape(dataset[i:i+target_size], (target_size, 1)))
        #labels.append(dataset[i:i+target_size])
    return np.array(data), np.array(labels)

In [9]:
train_set = np.genfromtxt("data/SEG_train_set.csv", delimiter="\n", dtype=np.int32)
x_train, y_train = generate_timeseries(train_set, 0, None, static_params["PAST_HISTORY"], static_params["FUTURE_TARGET"])
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.cache().batch(static_params["BATCH_SIZE"]).shuffle(static_params["BUFFER_SIZE"])

In [10]:
val_set = np.genfromtxt("data/SEG_val_set.csv", delimiter="\n", dtype=np.int32)
x_val, y_val = generate_timeseries(val_set, 0, None, static_params["PAST_HISTORY"], static_params["FUTURE_TARGET"])
val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_data = val_data.cache().batch(static_params["BATCH_SIZE"])

In [11]:
class SEGAutoRegressive(keras.Model):
    def __init__(self, units, dropout, out_steps):
        super().__init__()
        self.out_steps = out_steps
        self.units = units
        self.lstm_cell = keras.layers.LSTMCell(units, dropout=dropout)

        self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
        self.dense = Dense(static_params["VOCAB_SIZE"], activation="softmax")

    def warmup(self, inputs):
        onehot_inputs = tf.squeeze(tf.one_hot(inputs, static_params["VOCAB_SIZE"]), axis=2)

        # inputs.shape => (batch, time, features)
        # x.shape => (batch, lstm_units)
        x, *state = self.lstm_rnn(onehot_inputs)

        # predictions.shape => (batch, features)
        prediction = self.dense(x)

        return prediction, state

    def call(self, inputs, training=None):
        # Use a TensorArray to capture dynamically unrolled outputs.
        predictions = []
        # Initialize the lstm state
        prediction, state = self.warmup(inputs)

        # Insert the first prediction
        predictions.append(prediction)

        # Run the rest of the prediction steps
        for _ in range(self.out_steps - 1):
            # Use the last prediction as input.
            x = prediction

            # Execute one lstm step.
            x, state = self.lstm_cell(x, states=state, training=training)

            # Convert the lstm output to a prediction.
            prediction = self.dense(x)

            # Add the prediction to the output
            predictions.append(prediction)

        # predictions.shape => (time, batch, features)
        predictions = tf.stack(predictions)

        # predictions.shape => (batch, time, features)
        predictions = tf.transpose(predictions, [1, 0, 2])

        return predictions

In [18]:
model = SEGAutoRegressive(
    units=hparams["HP_LSTM_1_UNITS"], dropout=hparams["HP_LSTM_1_DROPOUT"], 
    out_steps=static_params["FUTURE_TARGET"])
model.compile(
    optimizer=keras.optimizers.Nadam(hparams["HP_LEARNING_RATE"]),
    loss="sparse_categorical_crossentropy",
    metrics=['accuracy']
)

In [19]:
with tf.summary.create_file_writer(log_dir).as_default():
    hp_api.hparams(hparams)
    history = model.fit(train_data, validation_data=val_data, epochs=500, callbacks=[
        keras.callbacks.EarlyStopping('val_accuracy', patience=5),
        keras.callbacks.TensorBoard(log_dir)
        ])
    #tf.summary.scalar('accuracy', accuracy, step=1)

...............] - ETA: 19s - loss: 0.6860 - accuracy: 0.776 81/158 [==============>...............] - ETA: 18s - loss: 0.6870 - accuracy: 0.776 82/158 [==============>...............] - ETA: 18s - loss: 0.6881 - accuracy: 0.776 83/158 [==============>...............] - ETA: 18s - loss: 0.6916 - accuracy: 0.774 84/158 [==============>...............] - ETA: 18s - loss: 0.6993 - accuracy: 0.771 85/158 [===============>..............] - ETA: 17s - loss: 0.7024 - accuracy: 0.769 86/158 [===============>..............] - ETA: 17s - loss: 0.7040 - accuracy: 0.769 87/158 [===============>..............] - ETA: 17s - loss: 0.7036 - accuracy: 0.769 88/158 [===============>..............] - ETA: 17s - loss: 0.7044 - accuracy: 0.769 89/158 [===============>..............] - ETA: 16s - loss: 0.7035 - accuracy: 0.770 90/158 [================>.............] - ETA: 16s - loss: 0.7049 - accuracy: 0.769 91/158 [================>.............] - ETA: 16s - loss: 0.7042 - accuracy: 0.769 92/158 [=======

In [27]:
model.save(version_dir)

INFO:tensorflow:Assets written to: version/20200917-120410\assets


In [28]:
model_1 = keras.models.load_model(version_dir) 


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.recurrent_v2.LSTMCell object at 0x0000018C8ADA9DC8> and <tensorflow.python.keras.layers.recurrent_v2.LSTMCell object at 0x0000018C8ADACD08>).


AssertionError: Some Python objects were not bound to checkpointed values, likely due to changes in the Python program: [<tf.Variable 'lstm_cell_1/kernel:0' shape=(16293, 512) dtype=float32, numpy=
array([[-0.00093962,  0.01685457, -0.00583172, ..., -0.00669129,
        -0.00767045, -0.00359495],
       [-0.00399957,  0.00933405, -0.00076441, ..., -0.01293601,
         0.01461778, -0.01425246],
       [ 0.00769919,  0.00736899, -0.01319111, ...,  0.00185939,
        -0.00596508, -0.00581026],
       ...,
       [-0.0133435 , -0.00114346,  0.01317015, ...,  0.00837968,
        -0.01000525,  0.00883356],
       [ 0.00267314,  0.00964596, -0.01599753, ...,  0.00599556,
         0.00175181,  0.01269117],
       [-0.00441695, -0.01714835, -0.00555437, ..., -0.01201999,
        -0.01785416, -0.01386872]], dtype=float32)>, <tf.Variable 'lstm_cell_1/bias:0' shape=(512,) dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0